In [10]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [11]:
# Load the GeoJSON file
gdf = gpd.read_file('data.geojson')

# Basic information about the dataset
print("Dataset Info:")
gdf.info()



Dataset Info:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4 entries, 0 to 3
Columns: 201 entries, OBJECTID to geometry
dtypes: float64(104), geometry(1), int32(96)
memory usage: 4.9 KB


In [ ]:
identity_columns = ["ZipCode"]
drop = ["OBJECTID"]
target = []

In [9]:
# Summary statistics of numerical columns
print("\nSummary Statistics:")
gdf.describe()




Summary Statistics:


,OBJECTID,ZipCode,Total_population,Male_Total_Population,Female_Total_Population,Percent_Male_Total_population__,Percent_Female_Total_population,Total_population_AGE_Under_5_ye,Total_population_AGE_5_to_9_yea,Total_population_AGE_10_to_14_y,...,Percent_Female_Total_populat_25,Percent_Female_Total_populat_26,Percent_Female_Total_populat_27,Percent_Female_Total_populat_28,Percent_Female_Total_populat_29,Percent_Female_Total_populat_30,Pop_by_Sq_Mile,SquareMiles,Shape__Area,Shape__Length
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.00000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000e+00,4.000000
mean,2.500000,85282.500000,46585.000000,24717.250000,21867.750000,52.588816,47.411184,1957.75000,1770.000000,1849.750000,...,83.700000,76.225000,19.225000,17.475000,13.400000,5.525000,4389.274718,10.250000,2.875398e+08,103118.556070
std,1.290994,1.290994,20640.118168,12014.679865,8924.902478,3.466592,3.466592,740.94731,665.803274,618.674591,...,4.988654,4.727491,7.397916,6.865069,4.910533,2.229163,1344.618598,2.217356,6.779669e+07,17899.151212
min,1.000000,85281.000000,19150.000000,10129.000000,9021.000000,48.512211,43.052621,916.00000,931.000000,946.000000,...,79.400000,70.800000,9.600000,8.700000,7.300000,3.400000,2393.750000,8.000000,2.187728e+08,79789.789707
25%,1.750000,85281.750000,39091.750000,19174.000000,19917.750000,51.130097,46.093442,1753.75000,1414.750000,1764.250000,...,79.925000,74.250000,15.975000,14.175000,11.050000,3.775000,4200.346591,8.750000,2.421862e+08,93812.698427
50%,2.500000,85282.500000,49283.500000,24830.500000,24453.000000,52.447838,47.552162,2131.50000,1848.500000,2052.000000,...,82.650000,75.900000,20.100000,18.200000,13.650000,5.350000,4942.328283,10.000000,2.795377e+08,107037.343828
75%,3.250000,85283.250000,56776.750000,30373.750000,26403.000000,53.906558,48.869903,2335.50000,2203.750000,2137.500000,...,86.425000,77.875000,23.350000,21.500000,16.000000,7.100000,5131.256410,11.500000,3.248912e+08,116343.201470
max,4.000000,85284.000000,68623.000000,39079.000000,29544.000000,56.947379,51.487789,2652.00000,2452.000000,2349.000000,...,90.100000,82.300000,27.100000,24.800000,19.000000,8.000000,5278.692308,13.000000,3.723110e+08,118609.746916


In [24]:
import geopandas as gpd
import pandas as pd
import numpy as np

def analyze_geojson(geojson_path, key_column):
    # Load the GeoJSON data into a GeoDataFrame
    gdf = gpd.read_file(geojson_path)
    
    # Print columns for debugging
    print("Columns in the GeoDataFrame:")
    print(gdf.columns)
    print(f"Key column: {key_column}")
    
    # Check if the key column exists in the GeoDataFrame
    if key_column not in gdf.columns:
        raise ValueError(f"Column '{key_column}' not found in the GeoDataFrame.")
    
    # Set the key column as the index if it's not already
    if gdf.index.name != key_column:
        gdf = gdf.set_index(key_column)
    
    # Separate geometry column if it exists
    if 'geometry' in gdf.columns:
        geometry = gdf['geometry']
        gdf = gdf.drop(columns=['geometry'])
    else:
        geometry = None
        print("Warning: No 'geometry' column found in the GeoDataFrame.")
    
    # Identify numeric columns
    numeric_columns = gdf.select_dtypes(include=[np.number]).columns.tolist()
    
    if not numeric_columns:
        raise ValueError("No numeric columns found in the GeoDataFrame.")
    
    # Perform basic analysis
    analysis = {}
    
    # Statistics for numeric columns
    for stat in ['max', 'min', 'mean', 'sum', 'median', 'std']:
        analysis[stat] = getattr(gdf, stat)()[numeric_columns]
    
    # Count of all columns
    analysis['count'] = gdf.count()
    
    # Additional analyses
    analysis['unique_counts'] = gdf.nunique()
    
    # Find the key (e.g., ZipCode) with highest and lowest values for each numeric column
    for col in numeric_columns:
        max_key = gdf[col].idxmax()
        min_key = gdf[col].idxmin()
        analysis[f'{col}_highest'] = f"{key_column}: {max_key} value: ({gdf[col].max()})"
        analysis[f'{col}_lowest'] = f"{key_column}: {min_key} value: ({gdf[col].min()})"
    
    return analysis, geometry

def print_analysis(analysis_results, key_column):
    for stat, result in analysis_results.items():
        if isinstance(result, pd.Series):
            print(f"{stat.capitalize()} values based on '{key_column}':")
            print(result.to_string())
        else:
            print(f"{stat}:")
            print(result)
        print("\n")

def main():
    geojson_path = "data.geojson"
    key_column = 'ZipCode'
    
    try:
        analysis_results, geometry = analyze_geojson(geojson_path, key_column)
        print_analysis(analysis_results, key_column)
        
        # Optional: You can use the geometry for further spatial analysis if needed
        if geometry is not None:
            print(f"Geometry column is available for spatial analysis. Shape: {geometry.shape}")
        else:
            print("No geometry column found in the GeoDataFrame.")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Columns in the GeoDataFrame:
Index(['OBJECTID', 'ZipCode', 'Total_population', 'Male_Total_Population',
       'Female_Total_Population', 'Percent_Male_Total_population__',
       'Percent_Female_Total_population', 'Total_population_AGE_Under_5_ye',
       'Total_population_AGE_5_to_9_yea', 'Total_population_AGE_10_to_14_y',
       ...
       'Percent_Female_Total_populat_26', 'Percent_Female_Total_populat_27',
       'Percent_Female_Total_populat_28', 'Percent_Female_Total_populat_29',
       'Percent_Female_Total_populat_30', 'Pop_by_Sq_Mile', 'SquareMiles',
       'Shape__Area', 'Shape__Length', 'geometry'],
      dtype='object', length=201)
Key column: ZipCode
Max values based on 'ZipCode':
OBJECTID                           4.000000e+00
Total_population                   6.862300e+04
Male_Total_Population              3.907900e+04
Female_Total_Population            2.954400e+04
Percent_Male_Total_population__    5.694738e+01
Percent_Female_Total_population    5.148779e+01
Total_po